<a href="https://colab.research.google.com/github/Rahuram841/game/blob/master/RawProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install spektral==0.6.0

     |████████████████████████████████| 95 kB 2.0 MB/s 


In [5]:
import numpy as np

import os
import networkx as nx
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import classification_report

from spektral.layers import GraphConv

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2

from collections import Counter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [50]:
from pandas import *
import csv
data = read_csv("/content/drive/MyDrive/spam (1) - spam (1).csv")
data=data.sample(frac = 1)
msgs = data['v2'].tolist()
labels=data['v1'].tolist()

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
pip install sentence_transformers

     |████████████████████████████████| 78 kB 2.9 MB/s 
     |████████████████████████████████| 3.4 MB 10.8 MB/s 
     |████████████████████████████████| 3.3 MB 40.0 MB/s 
     |████████████████████████████████| 1.2 MB 42.1 MB/s 
     |████████████████████████████████| 61 kB 454 kB/s 
     |████████████████████████████████| 596 kB 50.9 MB/s 
     |████████████████████████████████| 895 kB 52.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=70ccc579b50414f07454a1fad108d60b163b41bcc05fb22147bb3332a1eab974
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
sentence_embeddings = model.encode(msgs)

In [18]:
threshold=0.75

In [29]:
sentence_embeddings.shape


(5757, 768)

In [63]:
edges=[]
from sklearn.metrics.pairwise import cosine_similarity
for I in range(len(msgs)-1):
  values=cosine_similarity(
    [sentence_embeddings[I]],
    sentence_embeddings[I+1:]
  )
  for J in range(len(values[0])):
    if(values[0][J]>threshold):
       edges.append((str(I),str(I+J+1)))
print(edges)


[('0', '412'), ('0', '2417'), ('0', '2519'), ('1', '974'), ('1', '1000'), ('1', '1266'), ('1', '1275'), ('1', '1519'), ('1', '1523'), ('1', '1976'), ('1', '2374'), ('1', '3396'), ('1', '3750'), ('1', '4872'), ('1', '5063'), ('1', '5242'), ('1', '5390'), ('2', '113'), ('2', '166'), ('2', '311'), ('2', '334'), ('2', '417'), ('2', '504'), ('2', '524'), ('2', '563'), ('2', '877'), ('2', '1162'), ('2', '1462'), ('2', '1520'), ('2', '1535'), ('2', '1573'), ('2', '1766'), ('2', '2063'), ('2', '2123'), ('2', '2206'), ('2', '2310'), ('2', '2327'), ('2', '2426'), ('2', '2595'), ('2', '2631'), ('2', '2704'), ('2', '3162'), ('2', '3494'), ('2', '3673'), ('2', '3805'), ('2', '4009'), ('2', '4075'), ('2', '4694'), ('3', '1321'), ('3', '1651'), ('3', '1781'), ('3', '1891'), ('3', '2438'), ('3', '2867'), ('3', '2947'), ('3', '3240'), ('3', '3485'), ('3', '3603'), ('3', '3751'), ('3', '3836'), ('3', '4488'), ('3', '4562'), ('3', '4772'), ('3', '4974'), ('5', '551'), ('5', '672'), ('5', '1220'), ('5', '

In [64]:
msg_ids = [str(i) for i in range(len(msgs))]

In [65]:
msg_labels=labels

In [66]:
uwords =list()

In [67]:
for msg in msgs:
    for each in msg.split(" "):
        uwords.append(each)
uwords = set(uwords)

In [68]:
x_list = list()

In [69]:
for each in msgs:
    x_items = list()
    words = each.split(" ")
    for word in uwords:
        if word in words:
            x_items.append(1)
        else:
            x_items.append(0)
    x_list.append(x_items)

In [28]:
print(x_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(msg_labels)
print(x_list)
print(msg_ids)

In [70]:
X = np.array(x_list,dtype=int)
N = X.shape[0]
F = X.shape[1] 
num_classes = len(set(msg_labels))

In [98]:
train_idx = range(0,2500)
val_idx = range(2500,5000)
test_idx = range(5000,5757)

In [99]:
train_mask = np.zeros((N,),dtype=bool)
train_mask[train_idx] = True

val_mask = np.zeros((N,),dtype=bool)
val_mask[val_idx] = True

test_mask = np.zeros((N,),dtype=bool)
test_mask[test_idx] = True


In [100]:
def encode_label(labels):
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    labels = to_categorical(labels)
    return labels, label_encoder.classes_

labels_encoded, classes = encode_label(msg_labels)

In [101]:
nodes = msg_ids
edge_list = edges

In [102]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edge_list)

#obtain the adjacency matrix (A)
A = nx.adjacency_matrix(G)
print('Graph info: ', nx.info(G))

Graph info:  Graph with 5757 nodes and 54774 edges


In [ ]:
print(A)

In [125]:
channels = 16           # Number of channels in the first layer
dropout = 0.5           # Dropout rate for the features
l2_reg = 5e-4           # L2 regularization rate
learning_rate = 1e-2    # Learning rate
epochs = 20         # Number of training epochs
es_patience = 10        # Patience for early stopping

# Preprocessing operations
A = GraphConv.preprocess(A).astype('f4')

# Model definition
X_in = Input(shape=(F, ))
fltr_in = Input((N, ), sparse=True)

dropout_1 = Dropout(dropout)(X_in)
graph_conv_1 = GraphConv(channels,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         use_bias=False)([dropout_1, fltr_in])

dropout_2 = Dropout(dropout)(graph_conv_1)
graph_conv_2 = GraphConv(num_classes,
                         activation='softmax',
                         use_bias=False)([dropout_2, fltr_in])

# Build model
model = Model(inputs=[X_in, fltr_in], outputs=graph_conv_2)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()


Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 16097)]      0           []                               
                                                                                                  
 dropout_18 (Dropout)           (None, 16097)        0           ['input_19[0][0]']               
                                                                                                  
 input_20 (InputLayer)          [(None, 5757)]       0           []                               
                                                                                                  
 graph_conv_18 (GraphConv)      (None, 16)           257552      ['dropout_18[0][0]',             
                                                                  'input_20[0][0]']         

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
print(X.shape)
print(A)
print(labels_encoded.shape)

In [104]:
validation_data = ([X, A], labels_encoded, val_mask)
model.fit([X, A],
          labels_encoded,
          sample_weight=train_mask,
          epochs=epochs,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False,
          callbacks=[
#               EarlyStopping(patience=es_patience,  restore_best_weights=True),
          ])

Epoch 1/20
1/1 [==============================] - 5s 5s/step - loss: 0.8622 - acc: 0.0360 - val_loss: 0.8132 - val_acc: 0.8364
Epoch 2/20
1/1 [==============================] - 3s 3s/step - loss: 0.8138 - acc: 0.7420 - val_loss: 0.7691 - val_acc: 0.8432
Epoch 3/20
1/1 [==============================] - 3s 3s/step - loss: 0.7699 - acc: 0.7856 - val_loss: 0.7240 - val_acc: 0.8432
Epoch 4/20
1/1 [==============================] - 3s 3s/step - loss: 0.7250 - acc: 0.8036 - val_loss: 0.6764 - val_acc: 0.8436
Epoch 5/20
1/1 [==============================] - 3s 3s/step - loss: 0.6791 - acc: 0.8116 - val_loss: 0.6272 - val_acc: 0.8432
Epoch 6/20
1/1 [==============================] - 3s 3s/step - loss: 0.6332 - acc: 0.8144 - val_loss: 0.5787 - val_acc: 0.8424
Epoch 7/20
1/1 [==============================] - 3s 3s/step - loss: 0.5827 - acc: 0.8224 - val_loss: 0.5336 - val_acc: 0.8432
Epoch 8/20
1/1 [==============================] - 3s 3s/step - loss: 0.5367 - acc: 0.8308 - val_loss: 0.4936 - 

In [122]:
X_te = X[test_mask]
A_te = A[test_mask[0],:][:,test_mask[0]]

msg = input()
words = msg.split(" ")
items = list()
for word in uwords:
    if word in words:
        items.append(1)
    else:
        items.append(0)
items = np.array(items,dtype=int)
it = np.array(items).reshape(1,16097)
y_pred = model.predict([it, A_te], batch_size=N)
print(y_pred)
for each in y_pred:
    print(classes[np.argmax(each)])

why
[[0.14174584 0.14282802 0.14138246 0.1437661  0.14365183 0.14258455
  0.14404114]]
spam


In [123]:
bank_list=[]
for i in range(2500,5000):
  if labels[i]=='spam':
    bank_list.append(msgs[i])
    print(f"Contents:{msgs[i]}\n Lables:{labels[i]} ")
    print(len(bank_list))

Contents:our mobile number has won ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½5000, to claim calls us back or ring the claims hot line
 Lables:spam 
1
Contents:Warner Village 83118 C Colin Farrell in SWAT this wkend @Warner Village & get 1 free med. Popcorn!Just show msg+ticket@kiosk.Valid 4-7/12. C t&c @kiosk. Reply SONY 4 mre film offers
 Lables:spam 
2
Contents:Our records indicate u maybe entitled to 5000 pounds in compensation for the Accident you had. To claim 4 free reply with CLAIM to this msg. 2 stop txt STOP
 Lables:spam 
3
Contents:We have new local dates in your area - Lots of new people registered in YOUR AREA. Reply DATE to start now! 18 only www.flirtparty.us REPLYS150
 Lables:spam 
4
Contents:Spook up your mob with a Halloween collection of a logo & pic message plus a free eerie tone, txt CARD SPOOK to 8007 zed 08701417012150p per logo/pic
 Lables:spam 
5
Contents:Romantic Paris. 2 nights, 2 flights from ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½79 Book now 4 next year. Call 0870443
 Lables:spam 
6
Contents:Free Msg: 

In [126]:
X_te = X[test_mask]
A_te = A[test_mask[0],:][:,test_mask[0]]
for i in bank_list:
    msg = i
    words = msg.split(" ")
    items = list()
    for word in uwords:
      if word in words:
         items.append(1)
      else:
         items.append(0)
    items = np.array(items,dtype=int)
    it = np.array(items).reshape(1,16097)
    y_pred = model.predict([it, A_te], batch_size=N)
    #print(y_pred)
    for each in y_pred:
         print(f"Classes:{classes[np.argmax(each)]} Context:{i}" )

Classes:Ad Context:our mobile number has won ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½5000, to claim calls us back or ring the claims hot line
Classes:Ad Context:Warner Village 83118 C Colin Farrell in SWAT this wkend @Warner Village & get 1 free med. Popcorn!Just show msg+ticket@kiosk.Valid 4-7/12. C t&c @kiosk. Reply SONY 4 mre film offers
Classes:Recharge Context:Our records indicate u maybe entitled to 5000 pounds in compensation for the Accident you had. To claim 4 free reply with CLAIM to this msg. 2 stop txt STOP
Classes:ham Context:We have new local dates in your area - Lots of new people registered in YOUR AREA. Reply DATE to start now! 18 only www.flirtparty.us REPLYS150
Classes:ham Context:Spook up your mob with a Halloween collection of a logo & pic message plus a free eerie tone, txt CARD SPOOK to 8007 zed 08701417012150p per logo/pic
Classes:ham Context:Romantic Paris. 2 nights, 2 flights from ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½79 Book now 4 next year. Call 0870443
Classes:Recharge Context:Free Msg: Ringtone!Fr